### Imports

In [ ]:
from kub.course.simlib.simulation import FMUSimulation
from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.plotlib.weatherPlotFactory import WeatherPlotFactory
from pathlib import Path
import pandas as pd

repo_root = Path.cwd().parent.parent
databasePath = repo_root / "database" / "day2"

### **Materials**

##### Structural Materials (Heavy/Load-bearing)

In [ ]:
structural_materials = {
    "reinforced_concrete": {
        "lambda": 2.3,   # High conductivity (cold)
        "rho": 2400.0,   # Very heavy
        "Cp": 1000.0,    # Stores heat well
        "desc": "Standard reinforced concrete. Highly conductive but high thermal inertia."
    },
    "aerated_concrete": {
        "lambda": 0.15,  # Very insulating for a structural material
        "rho": 500.0,    # Light
        "Cp": 1000.0,
        "desc": "Cellular material (AAC/Siporex). A compromise between structure and insulation."
    },
    "solid_brick": {
        "lambda": 0.8,
        "rho": 1900.0,
        "Cp": 850.0,
        "desc": "Traditional solid red brick. High thermal inertia."
    },
    "hollow_brick": {
        "lambda": 0.35,
        "rho": 1000.0,
        "Cp": 900.0,
        "desc": "Standard honeycomb/perforated brick. Lighter than solid brick."
    },
    "hollow_concrete_block": {
        "lambda": 1.1,
        "rho": 1300.0,
        "Cp": 1000.0,
        "desc": "Classic cinder block (CMU). Poor insulation."
    },
    "granite_stone": {
        "lambda": 3.0,   # The most conductive!
        "rho": 2600.0,   # Very heavy
        "Cp": 800.0,
        "desc": "Hard stone, feels cold to the touch. Huge inertia."
    },
    "limestone": {
        "lambda": 1.4,
        "rho": 2200.0,
        "Cp": 850.0,
        "desc": "Soft stone (tuffeau/limestone). Slightly less cold than granite."
    },
    "rammed_earth": {
        "lambda": 0.85,
        "rho": 1900.0,
        "Cp": 1100.0,    # Excellent regulator
        "desc": "Compacted earth. Ecological with excellent inertia."
    },
    "oak_wood": {
        "lambda": 0.17,
        "rho": 750.0,
        "Cp": 1600.0,    # Very high Cp (characteristic of wood)
        "desc": "Hardwood, structural use."
    },
    "pine_wood": {
        "lambda": 0.13,  # Quite insulating for a structure
        "rho": 450.0,    # Light
        "Cp": 1600.0,
        "desc": "Light softwood. Typical for timber frame structures."
    }
}

#### Insulation Materials

In [ ]:
insulation_materials = {
    "glass_wool": {
        "lambda": 0.035, # Standard
        "rho": 15.0,     # Very light (low inertia)
        "Cp": 850.0,
        "desc": "Standard mineral/fiberglass wool. Very light, poor for thermal time lag."
    },
    "rock_wool": {
        "lambda": 0.038,
        "rho": 40.0,     # Slightly denser
        "Cp": 850.0,
        "desc": "Similar to glass wool but denser and fire resistant."
    },
    "expanded_polystyrene_EPS": {
        "lambda": 0.038,
        "rho": 20.0,
        "Cp": 1450.0,
        "desc": "Standard white foam. Insensitive to moisture."
    },
    "extruded_polystyrene_XPS": {
        "lambda": 0.032, # Better lambda
        "rho": 35.0,
        "Cp": 1450.0,
        "desc": "Dense blue/orange foam. Higher performance."
    },
    "polyurethane_PU": {
        "lambda": 0.024, # The champion of thermal insulation
        "rho": 32.0,
        "Cp": 1400.0,
        "desc": "PIR/PUR panel. Max insulation for minimal thickness."
    },
    "dense_wood_fiber": {
        "lambda": 0.042, # Lower insulation value...
        "rho": 160.0,    # ...but very heavy!
        "Cp": 2100.0,    # ...and huge Cp! (Great for summer comfort)
        "desc": "Rigid bio-based insulation. Excellent thermal time lag (phase shift)."
    },
    "cellulose_wadding": {
        "lambda": 0.040,
        "rho": 55.0,
        "Cp": 1900.0,
        "desc": "Recycled paper. Good compromise between performance/inertia/ecology."
    },
    "expanded_cork": {
        "lambda": 0.040,
        "rho": 110.0,
        "Cp": 1600.0,
        "desc": "Rot-proof, natural, good acoustic properties."
    },
    "straw_bale": {
        "lambda": 0.065, # Average lambda
        "rho": 90.0,
        "Cp": 1700.0,
        "desc": "Agricultural. Requires large thicknesses (35cm+)."
    },
    "aerogel": {
        "lambda": 0.015, # Sci-fi level (almost)
        "rho": 150.0,
        "Cp": 1000.0,
        "desc": "Ultra-high performance material (and ultra-expensive). Nanotechnology."
    }
}

### **Weather Display**

In [ ]:
# Load Weather Data
df_weather = pd.read_csv(databasePath / "YearlyWeather" / "weatherCity1_2024.csv")
df_weather['time'] = pd.to_datetime(df_weather['time'], unit='ms')
df_weather.set_index('time', inplace=True)

weather_factory = WeatherPlotFactory()

weather_factory.plot_yearly_temperature(df_weather)
# weather_factory.plot_yearly_solar_radiation(df_weather)

### **Building thermal simulation**

#### Functions

In [ ]:
#

def calculate_wall_resistance(Area, hi, he, material_layers):
    """
    Calculates the equivalent thermal resistance (Req) of a wall for N layers.

    Parameters:
    - Area (float): Surface area of the wall (m^2).
    - hi (float): Internal convective heat transfer coefficient (W/(m^2*K)).
    - he (float): External convective heat transfer coefficient (W/(m^2*K)).
                  NOTE: For ground/soil contact walls, 'he' should be set to 0.0
                  as the heat exchange model is different (handled via Delta_T).
    - material_layers (list of dict): List of the wall's layers.
      Each dict must contain 'e' (thickness in m) and 'lambda' (conductivity in W/(m*K)).

    Returns:
    - Req (float): The equivalent thermal resistance (K/W).
    """

    # 1. Internal Convective Resistance (R_hi)
    R_hi = 1 / (hi * Area) if (hi * Area) != 0 else 0

    # 2. Conduction Resistances (sum of N layers)
    R_total_conduction = 0.0
    for layer in material_layers:
        thickness = layer['e']
        lambda_mat = layer['lambda']
        # R_conduction = thickness / (lambda * Area)
        R_total_conduction += thickness / (lambda_mat * Area) if (lambda_mat * Area) != 0 else 0

    # 3. External Convective Resistance (R_he)
    R_he = 1 / (he * Area) if (he * Area) != 0 else 0

    # 4. Total Equivalent Resistance
    R_eq = R_hi + R_total_conduction + R_he

    return R_eq

In [ ]:
def calculate_total_heat_flow_balance(wall_balance_data, T_int, T_ext, airChange, L1, L2, H):
    """
    Calculates the heat flow (Q_flow) for each wall and the total flow, including ventilation.

    Parameters:
    - wall_balance_data (dict): Dictionary containing the necessary data for the balance.
      Each key is the wall name and the value is a dict
      containing 'Req' (Equivalent Resistance) and 'Delta_T' (T_int - T_target_surface).
    - T_int (float): Reference internal temperature (°C).
    - T_ext (float): Reference external temperature for ventilation (°C).
    - airChange (float): Air change rate (vol/s).
    - L1, L2, H (float): Building dimensions for volume calculation (m).

    Returns:
    - (dict): Individual flows, Q_walls_total, and Q_flow_total.
    """

    # --- Constants for Ventilation ---
    rho_air = 1.2    # Air density (kg/m^3)
    cp_air = 1006.0  # Specific heat capacity of air (J/(kg*K))

    # --- 1. Calculate Heat Loss through Walls ---
    results = {}
    Q_walls_total = 0.0

    for wall_name, data in wall_balance_data.items():
        R_eq = data['Req']
        Delta_T = data['Delta_T'] # Delta T supplied by the user (T_int - T_ext or T_int - T_soil)

        # Calculate Heat Flow (Q_flow = Delta_T / R_eq)
        if R_eq != 0:
            Q_flow = Delta_T / R_eq
        else:
            Q_flow = 0.0

        results[f"Q_flow_{wall_name}"] = Q_flow
        Q_walls_total += Q_flow

    # --- 2. Calculate Heat Loss through Ventilation ---
    Volume = L1 * L2 * H
    V_dot_air = airChange * Volume  # Volumetric flow rate (m^3/s)
    m_dot_air = rho_air * V_dot_air # Mass flow rate (kg/s)

    Delta_T_vent = T_int - T_ext # Ventilation is always between T_int and T_ext

    # Q_vent = m_dot * cp * Delta_T
    Q_vent = m_dot_air * cp_air * Delta_T_vent

    # --- 3. Final Result ---
    Q_flow_total = Q_walls_total + Q_vent

    results['Q_walls_total'] = Q_walls_total
    results['Q_vent'] = Q_vent
    results['Q_flow_total'] = Q_flow_total

    return results

#### Simulation and graph display

##### Names of variables to use in the code

Building Elements correspond to the **walls**, **roof** and the **ground slab** of the Building.

**Building Elements Thickness:**
- Wall
  - inner layer wall thickness : `eIntWall`
  - outer layer wall thickness : `eExtWall`
- Roof
  - inner layer roof thickness : `eIntRoof`
  - outer layer roof thickness : `eExtRoof`
- Ground Slab
  - inner layer bottom thickness : `eIntBottom`
  - outer layer bottom thickness : `eExtBottom`

**Materials properties of Building Elements:**
- Wall
  - thermal conductivity of inner layer (wall) : `lambdaIntWall`
  - thermal conductivity of outer layer (wall) : `lambdaExtWall`
- Roof
  - thermal conductivity of inner layer (roof) : `lambdaIntRoof`
  - thermal conductivity of outer layer (roof) : `lambdaExtRoof`
- Ground Slab
  - thermal conductivity of inner layer (bottom) : `lambdaIntBottom`
  - thermal conductivity of outer layer (bottom) : `lambdaExtBottom`

**Wall length:**
- length of walls 1 and 3 (symmetrical) : `LengthWall1`
- length of walls 2 and 4 (symmetrical) : `LengthWall2`

**Wall height:** `Height`

**Wall absorbtion:**
- absorption wall 1 : `absWall1`
- absorption wall 2 : `absWall2`
- absorption wall 3 : `absWall3`
- absorption wall 4 : `absWall4`

**Indoor air renewal:** `airchange`

**Sizing of heating and cooling capacity**
- heating power : `building.zone_habitable_1.Q_flow_heatingMax`
- cooling power : `building.zone_habitable_1.Q_flow_coolingMax`

In [ ]:
# input variables
inputs = {
    "eIntWall" : 1,
    "eExtWall" : 1,
    "eIntRoof" : 1,
    "eExtRoof" : 1,
    "eIntBottom" : 1,
    "eExtBottom" : 1,
    "lambdaIntWall" : 1,
    "lambdaExtWall" : 1,
    "lambdaIntRoof" : 1,
    "lambdaExtRoof" : 1,
    "lambdaIntBottom" : 1,
    "lambdaExtBottom" : 1,
    "LengthWall1" : 1,
    "LengthWall2" : 1,
    "Height" : 1,
    "absWall1" : 1,
    "absWall2" : 1,
    "absWall3" : 1,
    "absWall4" : 1,
    "airchange" : 1,
    "building.zone_habitable_1.Q_flow_heatingMax" : 1,
    "building.zone_habitable_1.Q_flow_coolingMax" : 1
}

In [ ]:
# variables to display
temperatureList = ["ambience.TAirRef", "building.zone_habitable_1.TAir"]
energyList = ["energyConsumptionHeating", "energyConsumptionCooling", "energyConsumptionTotal"]
convectiveCoeffList = ["Wall1_he", "Wall2_he", "Wall3_he", "Wall4_he", "Roof_he", "hi"]

In [ ]:
sim1 = FMUSimulation(databasePath / "BuildingModels" / "Exercices_Optimization_LoD0Bui4Walls1Floor1Roof_App4Walls1Floor1Roof.fmu")

sim1.initialize(startTime=0.0, stopTime=86400*365, timeStep=3600.0)
sim1.initParameters( {  } )
sim1.exitInitialization()

data1 = sim1.run( temperatureList + convectiveCoeffList + energyList )

sim1Factory = SimulationPlotFactory()

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="temperature",
    data_dict={ "Exterior Temperature (°C)": data1["ambience.TAirRef"],
                "Interior Temperature (°C)": data1["building.zone_habitable_1.TAir"] },
    title="Evolution of Temperature"
)

In [ ]:
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="raw",
    data_dict={ "Wall1_he": data1["Wall1_he"],
                "Wall2_he": data1["Wall2_he"],
                "Wall3_he": data1["Wall3_he"],
                "Wall4_he": data1["Wall4_he"],
                "Roof_he": data1["Roof_he"],
                "hi": data1["hi"] },
    title="Coefficient Convectif Exterieur"
)

In [ ]:
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="energy",
    data_dict={ "Cooling": data1["energyConsumptionCooling"],
                "Heating": data1["energyConsumptionHeating"],
                "Total": data1["energyConsumptionTotal"] },
    title="Energy Consumption over the year"
)